In [1]:
import cv2  # OpenCV for image processing
import numpy as np  # Numpy for numerical operations
import mss  # MSS for screen capture
import pyautogui  # PyAutoGUI for controlling the mouse
import time  # Time for delays
import keyboard  # Keyboard for detecting keypresses

def capture_screen():
    """
    Capture the entire screen and return it as an image array.
    """
    try:
        # Create a screen capture object
        with mss.mss() as sct:
            # Define the monitor to capture (1 refers to the primary monitor)
            monitor = sct.monitors[1]
            # Capture the screen as a raw image
            screenshot = sct.grab(monitor)
            # Convert the raw image to a numpy array
            screenshot = np.array(screenshot)
            # Convert from BGRA to BGR format
            screenshot = cv2.cvtColor(screenshot, cv2.COLOR_BGRA2BGR)
        return screenshot
    except Exception as e:
        # Print any error encountered during capture
        print(f"Error capturing screen: {e}")
        return None

def detect_sheep(screen, x, y, size=10):
    """
    Detect 'sheep' (specific color) in a given region of the screen.
    
    Parameters:
    screen: The captured screen image.
    x: The x-coordinate of the current mouse position.
    y: The y-coordinate of the current mouse position.
    size: The size of the region around (x, y) to search for 'sheep'.
    
    Returns:
    A tuple (detected_x, detected_y) if 'sheep' is detected, otherwise None.
    """
    if screen is None:
        return None
    
    height, width, _ = screen.shape
    if x < 0 or y < 0 or x >= width or y >= height:
        return None

    # Define the size of the region to search around the (x, y) position
    half_size = size // 2
    left = max(x - half_size, 0)
    top = max(y - half_size, 0)
    right = min(x + half_size, width)
    bottom = min(y + half_size, height)
    
    # Extract the region of interest from the screen
    region = screen[top:bottom, left:right]
    # Convert the region to HSV color space
    hsv = cv2.cvtColor(region, cv2.COLOR_BGR2HSV)

    # Define the color range for 'sheep' in HSV
    lower_sheep = np.array([0, 0, 180], dtype=np.uint8)
    upper_sheep = np.array([180, 30, 255], dtype=np.uint8)

    # Create a mask to identify the 'sheep' colors
    mask_sheep = cv2.inRange(hsv, lower_sheep, upper_sheep)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask_sheep, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Get the largest contour (likely to be the most significant 'sheep')
        largest_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 0:
            # Calculate the centroid of the largest contour
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                # Calculate the position of the detected 'sheep'
                detected_x = left + cx
                detected_y = top + cy
                return detected_x, detected_y
    return None

def main():
    """
    Main function that captures the screen, detects 'sheep', and performs actions based on detection.
    """
    running = False  # Initialize the running state as False
    
    while True:
        # Toggle the script's running state when 'p' is pressed
        if keyboard.is_pressed('p'):
            running = not running
            state = "ON" if running else "OFF"
            print(f"Script {state}")
            time.sleep(0.5)  # Delay to prevent multiple toggles from a single press
        
        if running:
            # Capture the current screen
            screen = capture_screen()
            if screen is None:
                print("Failed to capture screen.")
                time.sleep(1)
                continue
            
            # Get the current mouse position
            x, y = pyautogui.position()
            # Detect 'sheep' around the current mouse position
            detected_position = detect_sheep(screen, x, y)
            if detected_position:
                detected_x, detected_y = detected_position
                print(f"Detected sheep at: ({detected_x}, {detected_y})")
                # Move the mouse to the detected position and click
                pyautogui.moveTo(detected_x, detected_y)
                pyautogui.click()  # Perform a single click without delay
        
        # Short pause to reduce CPU usage
        time.sleep(0.01)

if __name__ == "__main__":
    main()


Script ON
Detected sheep at: (1189, 769)
Detected sheep at: (1188, 768)
Detected sheep at: (1187, 767)
Detected sheep at: (1209, 832)
Detected sheep at: (1219, 871)
Detected sheep at: (1218, 870)
Script OFF
Script ON
Detected sheep at: (994, 460)
Detected sheep at: (995, 459)
Detected sheep at: (997, 464)
Detected sheep at: (993, 462)
Detected sheep at: (993, 459)
Detected sheep at: (997, 463)
Detected sheep at: (997, 463)
Detected sheep at: (994, 464)
Detected sheep at: (950, 535)
Detected sheep at: (950, 535)
Detected sheep at: (949, 530)
Detected sheep at: (950, 535)
Detected sheep at: (946, 530)
Detected sheep at: (947, 529)
Detected sheep at: (952, 535)
Detected sheep at: (950, 535)
Detected sheep at: (950, 535)
Detected sheep at: (950, 535)
Detected sheep at: (950, 535)
Detected sheep at: (949, 530)
Detected sheep at: (950, 535)
Detected sheep at: (953, 529)
Detected sheep at: (950, 530)
Detected sheep at: (947, 530)
Detected sheep at: (954, 530)
Detected sheep at: (947, 530)
Det

KeyboardInterrupt: 